In [3]:
import os, json, csv, itertools
import numpy as np
import pandas as pd

from sklearn.preprocessing import normalize

In [22]:
# users = open("../MovieLens/basic_rounded/users.csv").read().split('\n')
users = open("users.txt").read().split('\n')

In [5]:
parent_dir = './'

results_files = ['BaselineOnly(SGD).txt',
'CoClustering.txt',
'KNNBasic(Itembased).txt',
'KNNBasic(Userbased).txt',
'KNNWithMeans(Itembased).txt',
'KNNWithMeans(Userbased).txt',
'NMF.txt',
'NormalPredictor.txt',
'SlopeOne.txt',
'SVD.txt']

mae_results = 'mae_results.json'
rmse_results = 'results.json'
_20m_results = '20m_results.json'

answer_matrix = 'oloco.csv'
time_matrix = 'time_matrix.csv'

In [5]:
len(results_files)

10

In [6]:
def read_results(root_dir):
    results = []
    for file in results_files:
        with open(os.path.join(root_dir, file)) as result:
            content = result.read().split('\n')[3:]
            results.append(json.loads(''.join(content)))
        result.close()
    return results

In [7]:
def get_time(root_dir):
    results = read_results(root_dir)
    time = []
    for user in users:
        time.append(list(map(lambda x: x[user]['time'], results)))
    return time

In [8]:
def write_file(filename, headers, rows):
    with open(os.path.join(parent_dir, filename), 'w', newline='') as file:
        file_writer = csv.writer(file)
        file_writer.writerow(headers)
        for row in rows:
            file_writer.writerow(row)
    file.close()

In [9]:
def without_scale(root_dir, metric='RMSE', results=None):
    if root_dir:
        results = read_results(root_dir)
    rmse = []
    for user in users:
        rmse.append(list(map(lambda x: x[user][metric], results)))
    return rmse

In [10]:
def unit_scale(root_dir, all_=False):
    results = read_results(root_dir)
    results_list = list(map(lambda x: list(map(lambda y: y[1]['RMSE'], x.items())), results))
    if all_:
        results_normalized = list(itertools.chain.from_iterable(results_list))
        results_normalized = 1-normalize([results_normalized])
        results_normalized = list(map(list, list(zip(*[iter(results_normalized[0])] * 200))))
    else:
        results_normalized = 1-normalize(results_list)
    return np.transpose(results_normalized)

In [11]:
def min_max_scale(root_dir, all_=True):
    results = read_results(root_dir)
    results_list = list(map(lambda x: list(map(lambda y: y[1]['RMSE'], x.items())), results))
    rmse = []
    if all_:
        big_list = list(itertools.chain.from_iterable(results_list))
        r_max = max(big_list)
        r_min = min(big_list)
        rmse = list(map(lambda x: (r_max - x)/(r_max - r_min), big_list))
    else:
        for result in results_list:
            r_max = max(result)
            r_min = min(result)
            rmse.append([(r_max - x)/(r_max - r_min) for x in result])
        rmse = list(itertools.chain.from_iterable(rmse))
    rmse = list(map(list, list(zip(*[iter(rmse)] * 200))))
    rmse = np.transpose(rmse)
    return rmse

In [23]:
def standard_scale(root_dir=None, metric='RMSE', results=None, n=300):
    if root_dir:
        results = read_results(root_dir)
        
    if results:        
        results_list = list(map(lambda x: list(map(lambda y: y[1][metric], x.items())), results))
        results_normalized = list(itertools.chain.from_iterable(results_list))
        results_normalized = list(map(lambda x: 1/(1+x), results_normalized))
        results_normalized = list(map(list, list(zip(*[iter(results_normalized)] * n))))        
        return np.transpose(results_normalized)
    
    return None

In [13]:
def standard_scale2(results, metric='mae', n=300):
    normal = list(map(lambda x: x[1][metric], results['NormalPredictor'].items()))
    results_list = []
    for alg, value in results.items():
        results_list_temp = list(map(lambda x: x[1][metric], value.items()))
        results_list.append(list(map(lambda x: 1/(1+(x[0]/x[1])), list(zip(results_list_temp, normal)) )))
        
    results_normalized = list(itertools.chain.from_iterable(results_list))
    results_normalized = list(map(list, list(zip(*[iter(results_normalized)] * n))))        
    return np.transpose(results_normalized)

In [14]:
# headers = [file.replace('.txt', '') for file in results_files]
# rmse = standard_scale(parent_dir)
# write_file(answer_matrix, headers, rmse)
# write_file(time_matrix, headers, time)

In [26]:
rmse_results_file = open(parent_dir+rmse_results)
results = json.load(rmse_results_file)
headers = list(results.keys())
rmse = standard_scale(metric='RMSE', results=[value for _, value in results.items()], n=500)
# mae = without_scale(root_dir=None, metric='mae', results=[value for _, value in results.items()])
write_file(answer_matrix, headers, rmse)

In [20]:
list(map(lambda x: list(map(lambda y: y[0], x.items())), [value for _, value in results.items()]))

[['1',
  '100',
  '101',
  '102',
  '103',
  '104',
  '105',
  '106',
  '107',
  '108',
  '11',
  '110',
  '111',
  '113',
  '114',
  '116',
  '117',
  '119',
  '12',
  '121',
  '123',
  '128',
  '129',
  '131',
  '132',
  '134',
  '136',
  '137',
  '139',
  '14',
  '140',
  '141',
  '142',
  '144',
  '146',
  '148',
  '149',
  '152',
  '153',
  '154',
  '155',
  '156',
  '16',
  '162',
  '165',
  '166',
  '167',
  '177',
  '178',
  '183',
  '19',
  '190',
  '191',
  '195',
  '196',
  '197',
  '202',
  '21',
  '211',
  '212',
  '213',
  '215',
  '22',
  '220',
  '221',
  '225',
  '227',
  '229',
  '230',
  '232',
  '233',
  '234',
  '235',
  '237',
  '238',
  '239',
  '242',
  '246',
  '247',
  '248',
  '249',
  '252',
  '253',
  '255',
  '256',
  '260',
  '261',
  '262',
  '268',
  '27',
  '271',
  '273',
  '274',
  '276',
  '279',
  '280',
  '285',
  '286',
  '287',
  '288',
  '29',
  '290',
  '291',
  '294',
  '298',
  '3',
  '300',
  '301',
  '302',
  '305',
  '306',
  '307',
  '30